In [1]:
####################################################### KOA  ##################################################################
import glob
import os
import pandas as pd
import numpy as np
import calendar
from contextlib import contextmanager
from datetime import datetime
import math
import warnings
warnings.filterwarnings("ignore")

In [4]:
os.chdir(r'C:\Users\becho\Documents\MasterReport\Reporting\Bases Manuales\11 FILL RATE\KOA')
FL = []
csv_files = glob.glob('*.xlsx')

In [5]:
for filename in csv_files:
    data = pd.read_excel(filename,sheet_name='Sheet1')
    data = data.drop([0,1],axis=0)
    FL.append(data)

In [6]:
FL = pd.concat(FL,ignore_index=True)
FL.rename(columns = {'Unnamed: 2': 'COD_CLI','Unnamed: 8':'SKU','Unnamed: 9':'PEDIDO','Unnamed: 3':'Fecha_t','Unnamed: 10':'DIF_PRECIO','Unnamed: 11':'OT_DIFERENCIA','Unnamed: 12':'QUIEBRE','Unnamed: 13':'CAPACIDAD','Unnamed: 14':'RECHAZO','Unnamed: 15':'SIN_LIQUIDAR'} ,inplace=True)
FL=FL[["Fecha_t","COD_CLI","SKU","PEDIDO","DIF_PRECIO","OT_DIFERENCIA","QUIEBRE","CAPACIDAD","RECHAZO","SIN_LIQUIDAR"]]

In [7]:
FL.PEDIDO = FL.PEDIDO.astype(float)
FL.PEDIDO = FL.PEDIDO.round(1)
FL.DIF_PRECIO = FL.DIF_PRECIO.astype(float)
FL.DIF_PRECIO = FL.DIF_PRECIO.round(1)
FL.OT_DIFERENCIA = FL.OT_DIFERENCIA.astype(float)
FL.OT_DIFERENCIA = FL.OT_DIFERENCIA.round(1)
FL.QUIEBRE = FL.QUIEBRE.astype(float)
FL.QUIEBRE = FL.QUIEBRE.round(1)
FL.CAPACIDAD = FL.CAPACIDAD.astype(float)
FL.CAPACIDAD = FL.CAPACIDAD.round(1)
FL.RECHAZO = FL.RECHAZO.astype(float)
FL.RECHAZO = FL.RECHAZO.round(1)
FL.SIN_LIQUIDAR = FL.SIN_LIQUIDAR.astype(float)
FL.SIN_LIQUIDAR = FL.SIN_LIQUIDAR.round(1)

In [8]:
FL_P=FL[["Fecha_t","COD_CLI","SKU","PEDIDO"]]
FL_P.rename(columns = {'PEDIDO':'VALOR'},inplace=True)
FL_P['Categoria']="Solicitado"

FL_DP=FL[["Fecha_t","COD_CLI","SKU","DIF_PRECIO"]]
FL_DP.rename(columns = {'DIF_PRECIO':'VALOR'},inplace=True)
FL_DP['Categoria']="Diferencia_de_precio"

FL_OT=FL[["Fecha_t","COD_CLI","SKU","OT_DIFERENCIA"]]
FL_OT.rename(columns = {'OT_DIFERENCIA':'VALOR'},inplace=True)
FL_OT['Categoria']="OT_DIFERENCIAS"

FL_Q=FL[["Fecha_t","COD_CLI","SKU","QUIEBRE"]]
FL_Q.rename(columns = {'QUIEBRE':'VALOR'},inplace=True)
FL_Q['Categoria']="Quiebre"

FL_C=FL[["Fecha_t","COD_CLI","SKU","CAPACIDAD"]]
FL_C.rename(columns = {'CAPACIDAD':'VALOR'},inplace=True)
FL_C['Categoria']="Capacidad"

FL_R=FL[["Fecha_t","COD_CLI","SKU","RECHAZO"]]
FL_R.rename(columns = {'RECHAZO':'VALOR'},inplace=True)
FL_R['Categoria']="Rechazo"

FL_SL=FL[["Fecha_t","COD_CLI","SKU","SIN_LIQUIDAR"]]
FL_SL.rename(columns = {'SIN_LIQUIDAR':'VALOR'},inplace=True)
FL_SL['Categoria']="Sin_liquidar"

In [9]:
FL_KOA = FL_DP.append(FL_P)
FL_KOA = FL_KOA.append(FL_OT)
FL_KOA = FL_KOA.append(FL_Q)
FL_KOA = FL_KOA.append(FL_C)
FL_KOA = FL_KOA.append(FL_R)
FL_KOA = FL_KOA.append(FL_SL)

In [10]:
os.chdir(r'C:\Users\becho\Documents\MasterReport\Maestras')
Maestro_SKU = pd.read_excel('Materiales  Abi.xlsx', sheet_name='Hoja1')#, dtype ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU.rename(columns = {'Material': 'SKU_KOA','Material ABI': 'Sku','HL/cj':'HL_cj'} ,inplace=True)

FL_KOA.rename(columns = {'SKU': 'Sku'} ,inplace=True)

Maestra_SKU_KOA = Maestro_SKU[["SKU_KOA","Sku"]]

In [11]:
FL_KOA =FL_KOA.dropna(subset=['Fecha_t'])
FL_KOA.Fecha_t = FL.Fecha_t.astype(str)

Titulo =FL_KOA[["Fecha_t"]]
Titulo = Titulo["Fecha_t"].str.split("-", n=2, expand=True)

aa=Titulo[2].str.split(" ", n=1, expand=True) #dia dd
Titulo[0] = Titulo[0].astype(str)             #anho yyyy
Titulo[1] = Titulo[1].astype(str)             #mes  mm
aa[0] = aa[0].astype(str)                     #dia str dd

FL_KOA['Fecha_t'] = Titulo[0]+Titulo[1]+aa[0] #yyyymmdd

FL_KOA =FL_KOA[["Fecha_t","COD_CLI", "Sku", "Categoria","VALOR"]]

In [12]:
################################################### KOE ####################################################################
os.chdir(r'C:\Users\becho\Documents\MasterReport\Reporting\Bases Manuales\11 FILL RATE\KOE\DATA')
FL_KOE = []
csv_files = glob.glob('*.csv')

In [13]:
for filename in csv_files:
    data = pd.read_csv(filename,encoding='UTF-16LE', sep='\t')#,dtype={'Stock_Cajas':np.float64, 'Sku_ABInBev': np.float64} ,decimal='.')
    FL_KOE.append(data)
FL_KOE = pd.concat(FL_KOE,ignore_index=True)

In [14]:
FL_KOE =FL_KOE[["Año_mes_dia_entrega","Codigo_Cliente", "Sku_ABInBev", "Cjs_Solicitadas", "ATRIBUTO","VALOR"]]
FL_KOE.VALOR = FL_KOE.VALOR.astype(float)
FL_KOE.VALOR = FL_KOE.VALOR.round(3)
FL_KOE.Cjs_Solicitadas = FL_KOE.Cjs_Solicitadas.astype(float)
FL_KOE.Cjs_Solicitadas = FL_KOE.Cjs_Solicitadas.round(3)

In [15]:
FL_SOL =FL_KOE[["Año_mes_dia_entrega","Codigo_Cliente", "Sku_ABInBev", "Cjs_Solicitadas", "ATRIBUTO"]]
FL_SOL = FL_SOL[(FL_SOL["ATRIBUTO"]=="MOTIVO_DIF_COSTO")]
FL_SOL['ATRIBUTO']="Solicitado"
FL_SOL.rename(columns = {'Cjs_Solicitadas': 'VALOR','Año_mes_dia_entrega':'Fecha_t','Codigo_Cliente':'COD_CLI','Sku_ABInBev':'Sku',} ,inplace=True)

In [16]:
FL_KOE.rename(columns = {'Año_mes_dia_entrega':'Fecha_t','Codigo_Cliente':'COD_CLI','Sku_ABInBev':'Sku'} ,inplace=True)
FL_KOE["ATRIBUTO"]=FL_KOE.ATRIBUTO.str.replace(r"MOTIVO_DIF_COSTO","Diferencia_de_precio")
FL_KOE["ATRIBUTO"]=FL_KOE.ATRIBUTO.str.replace(r"MOTIVO_ELIMINADAS","Eliminado")
FL_KOE["ATRIBUTO"]=FL_KOE.ATRIBUTO.str.replace(r"MOTIVO_ERROR_MAESTRA","ERROR_MAESTRA")
FL_KOE["ATRIBUTO"]=FL_KOE.ATRIBUTO.str.replace(r"MOTIVO_LIMITE_DE_CREDITO","Diferencia_de_precio")
FL_KOE["ATRIBUTO"]=FL_KOE.ATRIBUTO.str.replace(r"MOTIVO_QUIEBRE","Quiebre")
FL_KOE["ATRIBUTO"]=FL_KOE.ATRIBUTO.str.replace(r"MOTIVO_RETORNOS","Rechazo")
FL_KOE["ATRIBUTO"]=FL_KOE.ATRIBUTO.str.replace(r"MOTIVO_NO_IDENTIFICADAS","Otros")
FL_KOE = FL_KOE.append(FL_SOL)

In [17]:
FL_KOE.rename(columns = {'ATRIBUTO':'Categoria'} ,inplace=True)
FL_KOE =FL_KOE[["Fecha_t","COD_CLI", "Sku", "Categoria","VALOR"]]
FL_KOE = FL_KOE[FL_KOE["VALOR"]>0]

os.chdir(r'C:\Users\becho\Documents\MasterReport\Maestras')
Maestro_SKU = pd.read_excel('Materiales  Abi.xlsx', sheet_name='Hoja1')#, dtype ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU.rename(columns = {'Material': 'Sku_KOA','Material ABI': 'Sku','HL/cj':'HL_cj'} ,inplace=True)

FL_KOE.Fecha_t = FL_KOE.Fecha_t.astype(str)
FL_KOE.COD_CLI = FL_KOE.COD_CLI.astype(str)

pd.to_numeric(FL_KOE.Sku, errors='coerce',downcast='float')
FL_KOE.Sku = FL_KOE.Sku.astype(str)
Titulo =FL_KOE[["Sku"]]
Titulo = Titulo["Sku"].str.split(".", n=1, expand=True)
FL_KOE.Sku = Titulo[0]

FL_KOE =FL_KOE[["Fecha_t","COD_CLI", "Sku", "Categoria", "VALOR"]]
FL_KOA =FL_KOA[["Fecha_t","COD_CLI", "Sku", "Categoria", "VALOR"]]
FL_KOE['KO']="KOE"
FL_KOA['KO']="KOA"

FL_KOE.VALOR = FL_KOE.VALOR.astype(float)
FL_KOE.VALOR = FL_KOE.VALOR.round(4)

FL_KOE =FL_KOE.dropna(subset=['Fecha_t','COD_CLI', 'Sku', 'Categoria', 'VALOR'])
FL =FL_KOA.append(FL_KOE)

################################
FL = FL[FL["VALOR"]>0]
FL.VALOR = FL.VALOR.astype(float)
FL.VALOR = FL.VALOR.round(4)
FL =FL.dropna(subset=['Fecha_t','COD_CLI', 'Sku', 'Categoria', 'VALOR'])
#
FL =FL.dropna(subset=['Sku'])
FL = FL[FL["Sku"]!="nan"]

#############
FL.to_csv(r"C:\Users\becho\Documents\MasterReport\Reporting\ETL\07 Fill Rate\FILL_RATE.txt")

In [20]:
FLxx = FL[ FL["Fecha_t"]=="20210517" ]
FLxx = FLxx[FLxx["Categoria"]=="OT_DIFERENCIAS"]
FLxx.VALOR = FLxx.VALOR*-1
FL =FL.append(FLxx)

In [21]:
FLsolxx = FLxx
FLsolxx.Categoria ="Solicitado"
FL =FL.append(FLsolxx)

In [22]:
FLxx28 = FL[ FL["Fecha_t"]=="20210628" ]
FLxx28 = FLxx28[(FLxx28["Categoria"]=="OT_DIFERENCIAS")]
FLxx28.VALOR = FLxx28.VALOR*-1
FL =FL.append(FLxx28)

In [23]:
FLsolxx28 = FLxx28
FLsolxx28.Categoria ="Solicitado"
FL =FL.append(FLsolxx28)

In [24]:
FLxx29 = FL[ FL["Fecha_t"]=="20210629" ]
FLxx29 = FLxx29[(FLxx29["Categoria"]=="OT_DIFERENCIAS")]
FLxx29.VALOR = FLxx29.VALOR*-1
FL =FL.append(FLxx29)

In [25]:
FLsolxx29 = FLxx29
FLsolxx29.Categoria ="Solicitado"
FL =FL.append(FLsolxx29)

In [26]:
FLxx30 = FL[ FL["Fecha_t"]=="20210630" ]
FLxx30 = FLxx30[(FLxx30["Categoria"]=="OT_DIFERENCIAS")]
FLxx30.VALOR = FLxx30.VALOR*-1
FL =FL.append(FLxx30)

In [27]:
FLsolxx30 = FLxx30
FLsolxx30.Categoria ="Solicitado"
FL =FL.append(FLsolxx30)

In [28]:
FLxx06 = FL[ FL["Fecha_t"]=="20210906" ]
FLxx06 = FLxx06[(FLxx06["Categoria"]=="OT_DIFERENCIAS")]
FLxx06.VALOR = FLxx06.VALOR*-1
FL = FL.append(FLxx06)

In [29]:
FLsolxx06 = FLxx06
FLsolxx06.Categoria ="Solicitado"
FL = FL.append(FLsolxx06)

In [30]:
########################################   SI      #####################################
os.chdir(r'C:\Users\becho\Documents\MasterReport\Reporting\Bases Manuales\001 Sharepoint Jorge')
Sh_j = []
csv_files = glob.glob('*.txt')

for filename in csv_files:
    data = pd.read_csv(filename,encoding='ANSI', sep='\t')#,dtype={'Stock_Cajas':np.float64, 'Sku_ABInBev': np.float64} ,decimal='.')
    Sh_j.append(data)
Sh_j = pd.concat(Sh_j,ignore_index=True)

In [31]:
Sh_j = Sh_j[Sh_j["cga_tip_cga"]=="LOGISTICA VFILLRATE_TRADICSUPERM_DIARIO"]
Sh_j = Sh_j.dropna(subset=['log_anno_mes_dia_ent'])
Sh_j = Sh_j[['log_anno_mes_dia_ent', 'log_cod_cliente','log_sku_producto','log_mot_no_ent','log_cjs_sol','log_can_cjs_rech']]

In [32]:
########################################   SI      #####################################
os.chdir(r'C:\Users\becho\Documents\MasterReport\Reporting\Bases Manuales\001 Sharepoint Jorge')
Sh_j = []
csv_files = glob.glob('*.txt')

for filename in csv_files:
    data = pd.read_csv(filename,encoding='ANSI', sep='\t')#,dtype={'Stock_Cajas':np.float64, 'Sku_ABInBev': np.float64} ,decimal='.')
    Sh_j.append(data)
Sh_j = pd.concat(Sh_j,ignore_index=True)

In [33]:
Sh_j = Sh_j[Sh_j["cga_tip_cga"]=="LOGISTICA VFILLRATE_TRADICSUPERM_DIARIO"]
Sh_j = Sh_j.dropna(subset=['log_anno_mes_dia_ent'])
Sh_j = Sh_j[['log_anno_mes_dia_ent', 'log_cod_cliente','log_sku_producto','log_mot_no_ent','log_cjs_sol','log_can_cjs_rech']]

In [34]:
Sh_j.log_can_cjs_rech = Sh_j.log_can_cjs_rech.astype(float)
Sh_j.log_can_cjs_rech = Sh_j.log_can_cjs_rech.round(3)
Sh_j.log_cjs_sol = Sh_j.log_cjs_sol.astype(float)
Sh_j.log_cjs_sol = Sh_j.log_cjs_sol.round(3)
Sh_j.rename(columns = {'log_anno_mes_dia_ent':'Fecha_t','log_cod_cliente':'COD_CLI','log_sku_producto':'Sku_KOE','log_mot_no_ent':'ATRIBUTO','log_can_cjs_rech': 'VALOR'} ,inplace=True)

In [35]:
#BASE rechazos
FL_Sh = Sh_j
FL_Sh["ATRIBUTO"]=FL_Sh.ATRIBUTO.str.replace(r"MOTIVO_DIF_COSTO","Diferencia_de_precio")
FL_Sh["ATRIBUTO"]=FL_Sh.ATRIBUTO.str.replace(r"MOTIVO_ELIMINADAS","Eliminado")
FL_Sh["ATRIBUTO"]=FL_Sh.ATRIBUTO.str.replace(r"MOTIVO_ERROR_MAESTRA","ERROR_MAESTRA")
FL_Sh["ATRIBUTO"]=FL_Sh.ATRIBUTO.str.replace(r"MOTIVO_LIMITE_DE_CREDITO","Diferencia_de_precio")
FL_Sh["ATRIBUTO"]=FL_Sh.ATRIBUTO.str.replace(r"MOTIVO_QUIEBRE","Quiebre")
FL_Sh["ATRIBUTO"]=FL_Sh.ATRIBUTO.str.replace(r"MOTIVO_RETORNOS","Rechazo")
FL_Sh["ATRIBUTO"]=FL_Sh.ATRIBUTO.str.replace(r"MOTIVO_NO_IDENTIFICADAS","Otros")

In [36]:
FL_Sh.rename(columns = {'ATRIBUTO':'Categoria'} ,inplace=True)
FL_Sh =FL_Sh[["Fecha_t","COD_CLI", "Sku_KOE", "Categoria","VALOR"]]
FL_Sh = FL_Sh[FL_Sh["VALOR"]>0]
Sh_SOL = Sh_j
Sh_j = Sh_j[Sh_j["log_cjs_sol"]>0]

In [37]:
Sh_SOL.rename(columns = {'ATRIBUTO':'Categoria'} ,inplace=True)
Sh_SOL = Sh_SOL[(Sh_SOL["Categoria"]=="Eliminado")]
Sh_SOL.Categoria="Solicitado"
Sh_SOL =Sh_SOL[["Fecha_t","COD_CLI", "Sku_KOE", "Categoria","log_cjs_sol"]]
Sh_SOL.rename(columns = {'log_cjs_sol': 'VALOR'} ,inplace=True)
FL_Sh_c = FL_Sh.append(Sh_SOL)

In [38]:
os.chdir(r'C:\Users\becho\Documents\MasterReport\Reporting\Maestras')
Maestro_SKU = pd.read_excel('Materiales  Abi.xlsx', sheet_name='Hoja1')#, type ={'Cj/Pall simil':np.float64,'HL/cj':np.float64} ,decimal='.')
Maestro_SKU.rename(columns = {'SKU KOE': 'Sku_KOE','Material ABI': 'Sku','HL/cj':'HL_cj'} ,inplace=True)
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,5)
Maestro_SKU.HL_cj =Maestro_SKU.HL_cj/10
Maestro_SKU.HL_cj = np.round(Maestro_SKU.HL_cj,6)

In [39]:
FL_Sh_c2 = FL_Sh_c
Maestra_SKU_KOA = Maestro_SKU[["Sku_KOE","Sku"]]
Maestra_SKU_KOA = Maestra_SKU_KOA.drop_duplicates(subset=["Sku_KOE"])
FL_Sh_c2 = pd.merge(FL_Sh_c2,Maestra_SKU_KOA,on=["Sku_KOE"],how="outer")

In [41]:
FL_Sh_c2.Sku = FL_Sh_c2.Sku.astype(str)
FL_Sh_c2.Sku = FL_Sh_c2.Sku.str.split(".", n=1, expand=True)[0]

In [42]:
FL_Sh_c2.COD_CLI = FL_Sh_c2.COD_CLI.astype(str)
FL_Sh_c2.COD_CLI = FL_Sh_c2.COD_CLI.str.split(".", n=1, expand=True)[0]

In [44]:
FL_Sh_c2 = FL_Sh_c2.dropna(subset=['Sku'])

In [45]:
FL_Sh_c2 = FL_Sh_c2.dropna(subset=['Fecha_t','COD_CLI','Sku_KOE','Categoria','VALOR'])
FL_Sh_c2.to_csv(r"C:\Users\becho\Documents\MasterReport\Reporting\ETL\07 Fill Rate\FL_merg.txt")
FL_Sh_c2['KO']="KOE"

FL_Sh_c2['val'] = FL_Sh_c2.VALOR.astype(str)
FL_Sh_c2['Id'] = FL_Sh_c2.Fecha_t.astype(str) +"-"+ FL_Sh_c2.COD_CLI +"-"+FL_Sh_c2.Sku.astype(str)+"-"+FL_Sh_c2.Categoria.astype(str)+"-"+FL_Sh_c2.val
FL_Sh_c2 = FL_Sh_c2.drop_duplicates(subset=["Id"])

In [46]:
##### Error Cenco ###########
FL_Sh_c3 = (FL_Sh_c2[
                 ((FL_Sh_c2["Fecha_t"]==20210628) | (FL_Sh_c2["Fecha_t"]==20210629)| (FL_Sh_c2["Fecha_t"]==20210630))
                  & (FL_Sh_c2["Categoria"]=="Eliminado")
                    ])
FL_Sh_c3.VALOR = FL_Sh_c3.VALOR*-1
FL_Sh_c2 = FL_Sh_c2.append(FL_Sh_c3)

In [47]:
################################### DATA FINAL ###########################################
FL = FL.append(FL_Sh_c2)

FL = FL[["Fecha_t", "COD_CLI", "Sku", "Categoria", "VALOR", "KO"]]
FL = FL.dropna(subset=['Fecha_t','Sku',"Categoria", "VALOR"])
FL = FL.drop(FL[FL['Sku']=='nan'].index)
FL = FL.drop(FL[FL['Categoria']=='nan'].index)
FL = FL.drop(FL[FL['Fecha_t']=='nan'].index)

FL.to_csv(r"C:\Users\becho\Documents\MasterReport\Reporting\ETL\07 Fill Rate\FILL_RATE.txt")

In [ ]:
###################################### FIN #####################################